In [ ]:




import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import numpy as np  
import pandas as pd 
from numpy import ma
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import ticker, cm
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 

from keras import layers
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Embedding
from keras.utils import np_utils, to_categorical
from keras.datasets import imdb
#from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.optimizers import Adam

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, GlobalAveragePooling1D
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

import numpy as np
import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

sns.set(style='whitegrid', palette='muted')
rcParams['figure.figsize'] = 14, 8
np.random.seed(1)
tf.random.set_seed(1)

print('Tensorflow version:', tf.__version__)

In [ ]:
dataRaw = pd.read_csv('SampleInput.txt', sep='\s+', header=None)
RawTS = dataRaw.copy()
print(RawTS.shape)
RawTS.columns = ['year', 'day', 'hour', 'minute', 'second', 'millisecond', 'val1', 'val2', 'val3', 'val4', 'val5', 'val6', 'val7', 'val8', 'val9', 'val10', 'val11', 'val12', 'val13', 'val14', 'val15', 'val16', 'val17', 'val18', 'val19', 'val20', 'val21', 'val22']
RawTS.reset_index(inplace=True)
RawTS.rename(columns={'index': 'Serial Number'}, inplace=True)
# RawTS.head(50)


In [ ]:
def day_to_month_date(day):
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    # Check for leap year
    year = 2023
    if (year % 4 == 0 and year % 100 != 0) or year % 400 == 0:
        month_days[1] = 29
    
    # Find the corresponding month and date
    month = 0
    while day > month_days[month]:
        day -= month_days[month]
        month += 1
    
    return months[month], day

# # Example usage
day = 253
month, date = day_to_month_date(day)
print(f"{day} is {month} {date}")

In [ ]:
dates= []
months= []
for days in RawTS['day']:
    month, date = day_to_month_date(days)
    dates.append(date)
    months.append(month)

In [ ]:
RawTS.insert(loc=1, column='Month', value=months)
RawTS.insert(loc=1, column='Date', value=dates)

In [ ]:
new_col = RawTS['year'].astype(str) +"-"+ RawTS["Month"].astype(str)+"-"+ RawTS["Date"].astype(str) +" "+RawTS['hour'].astype(str) +":"+ RawTS["minute"].astype(str) +":"+RawTS['second'].astype(str) +"."+ RawTS["millisecond"].astype(str)
RawTS.insert(loc=1, column='TimeStamp', value=new_col)
new_col[1]

In [ ]:
RawTS.set_index(pd.to_datetime(RawTS['TimeStamp']), inplace=True)

In [ ]:
RawTS['val14'] = RawTS['val14'].replace({'SYNC': 1, 'NO_SYNC': 0})
RawTS['val15'] = RawTS['val15'].replace({'ON': 1, 'OFF': 0})
RawTS['val16'] = RawTS['val16'].replace({'ON': 1, 'OFF': 0})
RawTS['val17'] = RawTS['val17'].replace({'ON': 1, 'OFF': 0})
RawTS['val18'] = RawTS['val18'].replace({'SYNC': 1, 'NO_SYNC': 0})
RawTS['val19'] = RawTS['val19'].replace({'USBL': 1, 'NOT_USBL': 0})
RawTS['val22'] = RawTS['val22'].replace({'ON': 1, 'OFF': 0})
RawTS['val21'] = RawTS['val21'].replace({'LOW': 0, 'HIGH': 1})
RawTS['val20'] = RawTS['val22'].replace({'ENA': 1, 'DIS': 0})
# RawTS = RawTS.drop('val21', axis=1)
# RawTS = RawTS.drop('val19', axis=1)
RawTS = RawTS.drop('val1', axis=1)
RawTS = RawTS.drop('year', axis=1)
RawTS = RawTS.drop('day', axis=1)
RawTS = RawTS.drop('Month', axis=1)
RawTS = RawTS.drop('Date', axis=1)
RawTS = RawTS.drop('hour', axis=1)
RawTS = RawTS.drop('minute', axis=1)
RawTS = RawTS.drop('second', axis=1)
RawTS = RawTS.drop('millisecond', axis=1)

#RawTS['val1'] = RawTS['val1'].replace(RawTS['val1']/4)
RawTS.head()


In [ ]:
RawTS.head()

In [ ]:
for i in range(2,23):
    RawTS[f'val{i}']=RawTS[f'val{i}'].astype(float)

In [ ]:
labels = pd.read_csv('SampleInput.txt', sep='\s+', header=None)
print(labels.shape)
labels.head()

In [ ]:
labels.value_counts(), labels.value_counts(normalize=True)

In [ ]:
RawTS.head()

In [ ]:
for i in range(2,23):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=RawTS['TimeStamp'], y=RawTS[f'val{i}'],name=f'VAL{i}'))
    fig.update_layout(showlegend=True)
    fig.show()

In [ ]:
train_size = int(len(RawTS) * 0.7)
test_size = len(RawTS) - train_size
train, test = RawTS.iloc[0:train_size], RawTS.iloc[train_size:len(RawTS)]
print(train.shape, test.shape)

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
anomaly={}
for i in range(2,23):
    time_steps = 30

    X_train, y_train = create_dataset(train[[f'val{i}']], train[f'val{i}'], time_steps)
    X_test, y_test = create_dataset(test[[f'val{i}']], test[f'val{i}'], time_steps)
    np.random.seed(21)
    tf.random.set_seed(21)
    timesteps = X_train.shape[1]
    num_features = X_train.shape[2]
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

    model = Sequential([
        LSTM(128, input_shape=(timesteps, num_features)),
        Dropout(0.2),
        RepeatVector(timesteps),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        TimeDistributed(Dense(num_features))                 
    ])

    model.compile(loss='mae', optimizer='adam')
    model.summary()
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')
    history = model.fit(
        X_train, y_train,
        epochs=5,
        batch_size=32,
        validation_split=0.1,
        callbacks = [es],
        shuffle=False
    )
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend();
    X_train_pred = model.predict(X_train)
    train_mae_loss = pd.DataFrame(np.mean(np.abs(X_train_pred - X_train), axis=1), columns=['Error'])
    # Mean Absolute Error loss
    X_train_pred = model.predict(X_train)
    train_mae_loss = np.mean(np.abs(X_train_pred - X_train), axis=1)

    plt.hist(train_mae_loss, bins=50)
    plt.xlabel('Train MAE loss')
    plt.ylabel('Number of Samples');

    # Set reconstruction error threshold
    threshold = np.max(train_mae_loss)

    print('Reconstruction error threshold:',threshold)
    model.evaluate(X_test, y_test)
    sns.distplot(train_mae_loss, bins=50, kde=True);
    X_test_pred = model.predict(X_test)

    test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)
    sns.distplot(test_mae_loss, bins=50, kde=True);
#     threshold = 0.23
    anomaly_df = pd.DataFrame(test[30:])
    anomaly_df['loss'] = test_mae_loss
    anomaly_df['threshold'] = threshold
    anomaly_df['anomaly'] = anomaly_df['loss'] > anomaly_df['threshold']
    anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
    anomaly[f'anomaly_var{i}'] = anomalies
#     anomaly_df.head()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=anomaly_df['TimeStamp'], y=anomaly_df['loss'], name='Test loss'))
    fig.add_trace(go.Scatter(x=anomaly_df['TimeStamp'], y=anomaly_df['threshold'], name='Threshold'))
    fig.update_layout(showlegend=True, title='Test loss vs. Threshold')
    fig.show()